Note: This a notebook in progress

# Getting Data 

Here I am importing BDTNS data that has been filtered to include Drehem data

In [134]:
import pandas as pd
PATH = "filtered.csv"
filtered = pd.read_csv(PATH)

In [135]:
filtered.head()

,Unnamed: 0,lemma,id_text,id_line,id_word,label
0,0,3(diš)[]NU,epsd2/admin/u3adm/P124149,3,P124149.3.1,o 1
1,1,udu[sheep]N,epsd2/admin/u3adm/P124149,3,P124149.3.2,o 1
2,2,6(diš)[]NU,epsd2/admin/u3adm/P124149,4,P124149.4.1,o 2
3,3,u[ewe]N,epsd2/admin/u3adm/P124149,4,P124149.4.2,o 2
4,4,2(diš)[]NU,epsd2/admin/u3adm/P124149,5,P124149.5.1,o 3


# Identifying People

Here, I create a dictionary with all the PN as keys. The values are dictionaries that contain profession and role mappings. In the future, I will add professions and roles to the PNs. The goal of this notebook is to finally have a csv with three columns for PN, associated role, and associated profession respectively. 

In [111]:
people = dict()
for row in filtered.itertuples():
    if 'PN' in row.lemma:
        people[(row.lemma, row.id_word)] = dict()

In [112]:
#sanity check
#print(people.keys())

# Generate List of Professions Based on Location

Here I have a list of professions that was retrieved from external sources:.

SOURCE: URAP meeting notes

In [113]:

professions = [ "aʾigidu[worker]", 
                "abala[water-drawer]", 
                "abrig[functionary]", 
                "ad.KID[weaver]", 
                "agaʾus[soldier]",
                "arad[slave]",
                "ašgab[leatherworker]",
                "aʾua [musician]", 
                "azlag[fuller]",
                "bahar[potter]",
                "bisaŋdubak[archivist]",
                "damgar[merchant]",
                "dikud[judge]",
                "dubsar[scribe]",
                "en[priest]",
                "erešdiŋir[priestess]",
                "ensik[ruler]",
                "engar[farmer]",
                "enkud[tax-collector]",
                "gabaʾaš[courier]",
                "galamah[singer]",
                "gala[singer]",
                "geme[worker]",
                "gudug[priest]",
                "guzala[official]",
                "idu[doorkeeper]",
                "išib[priest]",
                "kaguruk[supervisor]",
                "kaš[runner]",
                "kiŋgia[messenger]",
                "kinda[barber]", 
                "kinkin[miller]",
                "kiridab[driver]", 
                "kurušda[fattener]", 
                "kuš[official]",
                "lu[person]",
                "lugal[king]",
                "lukur[priestess]",
                "lungak[brewer]",
                "malah[sailor]",
                "muhaldim[cook]",
                "mušendu[bird-catcher]",
                "nagada[herdsman]",
                "nagar[carpenter]",
                "nar[musician]",
                "nargal[musician]", 
                "narsa[musician]", 
                "nin[lady]",
                "nubanda[overseer]",
                "nukirik[horticulturalist]",
                "saŋ.DUN₃[recorder]",
                "saŋŋa[official]",
                "simug[smith]",
                "sipad[shepherd]",
                "sukkal[secretary]",
                "šabra[administrator]",
                "šagia[cup-bearer]",
                "šakkanak[general]",
                # "szej[cook]", this is a verb
                "šidim[builder]",
                "šuʾi[barber]",
                "šukud[fisherman]",
                "tibira[sculptor]",
                "ugula[overseer]",
                "unud[cowherd]",
                # "urin[guard]",
                "UN.IL₂[menial]",
                "ušbar[weaver]",
                "zabardab[official]",
                "zadim[stone-cutter]"]

profession_counts = { i : 0 for i in professions }

If previous word is a 'PN' and the current word matches a profession, I'll add that profession to the PN. The commented out parts are from when I tried to generate my own list of professions. 

In [114]:
import re

prev_row = None
for row in filtered.itertuples():
        if prev_row != None and 'PN' in prev_row.lemma and (re.match('^[^\]]*', row.lemma)[0] + "]") in professions:
                profession_counts[(re.match('^[^\]]*', row.lemma)[0] + "]")] += 1
                people[(prev_row.lemma, prev_row.id_word)]['profession'] = row.lemma
        prev_row = row
        
        
english = set()

#for word in words:
    #to get only english use this one
    #english.append((re.search('\[(.*?)\]', str(word)).group()))
 #   english.add(word[0])

#print(len(english))

#for val in set(english):
 #   print(val[1:-1])

#Since this is related to the n-neighbors, this CSV is later on used in the neighbors notebook
pd.DataFrame(list(english)).to_csv('words_after_PN.csv', index = False)

In [115]:
pd.DataFrame.from_dict(profession_counts, orient='index', columns=['Percentages']).head()

,Percentages
aʾigidu[worker],0
abala[water-drawer],0
abrig[functionary],0
ad.KID[weaver],1
agaʾus[soldier],55


In [116]:
total = sum(profession_counts.values())
profession_percentages = { i : profession_counts[i]/total for i in profession_counts}
percentage_df = pd.DataFrame.from_dict(profession_percentages, orient='index', columns=['Percentage'])
percentage_df.head(15)

,Percentage
aʾigidu[worker],0.000000
abala[water-drawer],0.000000
abrig[functionary],0.000000
ad.KID[weaver],0.000100
agaʾus[soldier],0.005486
arad[slave],0.037702
ašgab[leatherworker],0.000199
aʾua [musician],0.000000
azlag[fuller],0.000100
bahar[potter],0.000000


Looking at professions that never appear:

In [117]:
percentage_df[percentage_df['Percentage'] ==  0.0]


,Percentage
aʾigidu[worker],0.0
abala[water-drawer],0.0
abrig[functionary],0.0
aʾua [musician],0.0
bahar[potter],0.0
gabaʾaš[courier],0.0
kiŋgia[messenger],0.0
kinda[barber],0.0
kinkin[miller],0.0
kiridab[driver],0.0


In [118]:
percentage_df.sort_values(by='Percentage', ascending=False)

,Percentage
dubsar[scribe],0.196689
lugal[king],0.168761
ensik[ruler],0.067923
sukkal[secretary],0.062438
kurušda[fattener],0.059944
lu[person],0.055157
šagia[cup-bearer],0.049072
šabra[administrator],0.045781
arad[slave],0.037702
nubanda[overseer],0.036705


# Finding Roles

More information on keywords can be found here:
https://github.com/niekveldhuis/Sumerian-network/blob/master/Finding%20Merging%20Roles%20and%20Professions/roles_and_keywords.md 

In [119]:
import re
def match_ki_ta():
    prev_row = None
    i=0
    words = []
    for row in filtered.itertuples():
        if re.match(r".+ta\[\]PN", row.lemma) and 'ki[place]N' in prev_row:
            words.append((row.lemma, row.id_word))
            i+=1
        prev_row = row
    print('Number of roles found:', i)
    return (words, i)


def find_keyword_prev(keyword):
    prev_row = None
    i=0
    words = []
    for row in filtered.itertuples():
        if prev_row != None and keyword in row.lemma and 'PN' in prev_row.lemma:
            words.append((prev_row.lemma, prev_row.id_word))
            i+=1
        prev_row = row
    print('Number of roles found:', i)
    return (words, i)
    
def find_keyword_next(keyword):
    prev_row = None
    i=0
    words = []
    for row in filtered.itertuples():
        if prev_row != None and keyword in prev_row.lemma and 'PN' in row.lemma:
            words.append((row.lemma, row.id_word))
            i+=1
        prev_row = row
    print('Number of roles found:', i)
    return (words, i)

Creating a dictionary that maps roles to count of those roles

In [120]:
role_counts = dict()

1. Matching rows that have 'ki[place]N' in previous cell and '-ta' in current

In [121]:
#source
for person, id_w in match_ki_ta()[0]:
    try:
        people[(person, id_w)]['role'] = 'source'
    except:
        print(person, id_w)
        
role_counts['source'] = match_ki_ta()[1]

Number of roles found: 9284
Number of roles found: 9284


2. Matching rows that have 'dab[seize]V/t' in lemmatization

In [122]:
res = find_keyword_prev('dab[seize]V/t')
for (person, id_w) in res[0]:
    try:
        people[(person, id_w)]['role'] = 'recipient'
    except:
        print(person, id_w)
        
role_counts['recipient'] = res[1]

Number of roles found: 5102


3. Matching rows that have 'mu.DU[delivery]N' in next cell

In [123]:
# new owner
res = find_keyword_next('mu.DU[delivery]N')
for (person, id_w) in res[0]:
    try:
        people[(person, id_w)]['role'] = 'new owner'
    except:
        print((person, id_w))
        
role_counts['new owner'] = res[1]

Number of roles found: 2304


4. Matching rows that have 'šu[hand]N' in next cell

In [124]:
res = find_keyword_prev('šu[hand]N')
for (person, id_w) in res[0]:
    try:
        people[(person, id_w)]['role'] = 'recipient'
    except:
        print(person, id_w)
role_counts['recipient'] += res[1]

Number of roles found: 2594


5. Matching rows that have 'ŋiri[foot]N' in next cell

In [125]:
res = find_keyword_next('ŋiri[foot]N')
for (person, id_w) in res[0]:
    try:
        people[(person, id_w)]['role'] = 'intermediary'
    except:
        print((person, id_w))
        
role_counts['intermediary'] = res[1]        

Number of roles found: 3154


6. Looks for rows with 'maškim' in previous spot

In [126]:
res = find_keyword_prev('maškim[administrator]N')
for (person, id_w) in res[0]:
    try:
        people[(person, id_w)]['role'] = 'representative'
    except:
        print((person, id_w))
        
role_counts['representative'] = res[1]      

Number of roles found: 2072


7. Looking for PNs before and after 'zig[rise]V/i'

In [127]:
res = find_keyword_next('zig[rise]V/i')
for (person, id_w) in res[0]:
    try:
        people[(person, id_w)]['role'] = 'source'
    except:
        print((person, id_w))
        
role_counts['source'] += res[1]

Number of roles found: 12


In [128]:
res = find_keyword_prev('zig[rise]V/i')
for (person, id_w) in res[0]:
    try:
        people[(person, id_w)]['role'] = 'source'
    except:
        print((person, id_w))
        
role_counts['source'] += res[1]

Number of roles found: 2383


This is every PN mentioned in the filtered texts

In [129]:
len(people.keys())

46126

In [130]:
people_df = pd.DataFrame(list(people.items()))
people_df.head(4)

,0,1
0,"({d}šul-gi-a-a-mu-ta[]PN, P405489.12.2)",{'role': 'source'}
1,"({d}šul-gi-iri-mu[]PN, P405489.13.1)",{'role': 'recipient'}
2,"({d}utu-ba-ni[]PN, P332036.11.1)",{}
3,"(dingir-ki-ma-at[]PN, P332036.19.3)",{}


# Transfer to CSV

In [139]:
name_word = pd.DataFrame(people_df[0].values.tolist(), index=people_df.index, columns = ['Name', 'id_word'])
final = name_word.join(people_df.drop(people_df.columns[0], axis = 1))



In [140]:
final = pd.concat([final.drop([1], axis=1), final[1].apply(pd.Series)], axis=1)

In [141]:
final.head()

,Name,id_word,role,profession
0,{d}šul-gi-a-a-mu-ta[]PN,P405489.12.2,source,NaN
1,{d}šul-gi-iri-mu[]PN,P405489.13.1,recipient,NaN
2,{d}utu-ba-ni[]PN,P332036.11.1,NaN,NaN
3,dingir-ki-ma-at[]PN,P332036.19.3,NaN,NaN
4,di-ku₅-i₃-li₂[]PN,P332036.19.6,NaN,NaN


adding CDLI No column

In [142]:
CDLI_No = final['id_word'].apply(lambda x: re.split('\.', x)[0])
final.insert(4, 'CDLI No', CDLI_No)
final.head()

,Name,id_word,role,profession,CDLI No
0,{d}šul-gi-a-a-mu-ta[]PN,P405489.12.2,source,NaN,P405489
1,{d}šul-gi-iri-mu[]PN,P405489.13.1,recipient,NaN,P405489
2,{d}utu-ba-ni[]PN,P332036.11.1,NaN,NaN,P332036
3,dingir-ki-ma-at[]PN,P332036.19.3,NaN,NaN,P332036
4,di-ku₅-i₃-li₂[]PN,P332036.19.6,NaN,NaN,P332036


In [154]:

final.to_csv('roles_professions.csv')

In [144]:
#TODO: need to deal with case when PN not right next to role
[(row.lemma, row.id_word) for row in filtered.itertuples() if 'P330639.17' in row.id_word]

[('a-hu-ni[]PN', 'P330639.17.1'),
 ('šagia[cup-bearer]N', 'P330639.17.2'),
 ('maškim[administrator]N', 'P330639.17.3')]

# Analyzing Roles and Professions

Here we analyze percentages of roles. Previously we created a dataframe for percentages of professions

In [145]:
#role percentages

total = sum(role_counts.values())
role_counts.update((k,role_counts[k]/total) for k in role_counts)
print(role_counts)
pd.DataFrame.from_dict(role_counts, orient='index', columns=['Percentages'])

{'source': 0.43408288422226354, 'recipient': 0.2860434863408288, 'new owner': 0.08563464040141237, 'intermediary': 0.11722728117450289, 'representative': 0.07701170786099239}


,Percentages
source,0.434083
recipient,0.286043
new owner,0.085635
intermediary,0.117227
representative,0.077012


# Merging with BDTNS Dates

In [146]:
dates_file = 'bdtns_metadata/dates.csv'
dates_df = pd.read_csv(dates_file)
dates_df.head()

,Unnamed: 0,BDTNS No,CDLI No,Original date,Converted Date
0,0,38699,P142785,IS01 - 07 - 00,85.0700
1,1,38700,P142787,SS05 - 06 - 12,80.0612
2,2,38701,P142788,SH48 - 12 - 20,66.1220
3,3,38702,P142789,SS04 - XX - 15,79.0015
4,4,38703,P142790,XXXX - 01 - 01,0.0101


In [147]:
final.head()

,Name,id_word,role,profession,CDLI No
0,{d}šul-gi-a-a-mu-ta[]PN,P405489.12.2,source,NaN,P405489
1,{d}šul-gi-iri-mu[]PN,P405489.13.1,recipient,NaN,P405489
2,{d}utu-ba-ni[]PN,P332036.11.1,NaN,NaN,P332036
3,dingir-ki-ma-at[]PN,P332036.19.3,NaN,NaN,P332036
4,di-ku₅-i₃-li₂[]PN,P332036.19.6,NaN,NaN,P332036


In [150]:
merged = dates_df.merge(final)

In [153]:
#rearranging columns to make it easier to use as a node list
merged_df = merged[['Name', 'id_word', 'CDLI No', 'role', 'profession', 'Original date', 'Converted Date']]
merged_df

,Name,id_word,CDLI No,role,profession,Original date,Converted Date
0,ur-{d}dam-gal-nun-na[]PN,P142785.4.1,P142785,NaN,dubsar[scribe]N,IS01 - 07 - 00,85.0700
1,ur-{d}šul-pa-e₃[]PN,P142785.6.1,P142785,NaN,dubsar[scribe]N,IS01 - 07 - 00,85.0700
2,{d}šul-gi-i₃-li₂[]PN,P142785.16.1,P142785,recipient,NaN,IS01 - 07 - 00,85.0700
3,lu₂-giri₁₇-zal[]PN,P142785.17.2,P142785,intermediary,NaN,IS01 - 07 - 00,85.0700
4,{d}i-bi₂-{d}suen[]PN,P142785.19.2,P142785,NaN,lugal[king]N,IS01 - 07 - 00,85.0700
5,ur-{d}dam-gal-nun-na[]PN,P142785.23.1,P142785,NaN,dubsar[scribe]N,IS01 - 07 - 00,85.0700
6,ur-{d}šul-pa-e₃[]PN,P142785.25.1,P142785,NaN,dubsar[scribe]N,IS01 - 07 - 00,85.0700
7,{d}šul-gi-i₃-li₂[]PN,P142785.35.1,P142785,new owner,NaN,IS01 - 07 - 00,85.0700
8,lu₂-giri₁₇-zal[]PN,P142785.36.2,P142785,intermediary,NaN,IS01 - 07 - 00,85.0700
9,{d}i-bi₂-{d}suen[]PN,P142785.39.2,P142785,NaN,lugal[king]N,IS01 - 07 - 00,85.0700


In [158]:

merged_df.to_csv('Graph Representation/nov_2019_nodelist.csv')